### Kaustav Vats (2016048)

In [12]:
import numpy as np
import sys
import copy
from apyori import create_next_candidates, TransactionManager
from itertools import combinations
from tqdm import tqdm_notebook as tqdm
import time
import matplotlib.pyplot as plt
%matplotlib inline 
# https://github.com/ymoch/apyori/blob/master/apyori.py
# https://www.geeksforgeeks.org/itertools-combinations-module-python-print-possible-combinations/

In [3]:
def LoadData(filename):
    d = []
    f = open("Data/" + filename, 'r')
    for line in f:
        if line == "":
            break
        d.append(list(map(int, line.split("\n")[0].split())))
    return d

def GetMax(data):
    maxi = -1
    mini = sys.maxsize
    for t in data:
        maxi2 = max(t)
        mini2 = min(t)
        if maxi2 > maxi:
            maxi = maxi2
        if mini2 < mini:
            mini = mini2
    return maxi, mini

def GetSupport(N, data, mini, maxi):
    s = {}
    for i in range(N):
        for j in range(len(data[i])):
            if data[i][j] in s:
                s[data[i][j]] += 1
            else:
                s[data[i][j]] = 1
    for k in s:
        s[k] = s[k]/N
    return s

def GetC1(s, min_supp):
    c1 = copy.deepcopy(s)
    for k in s:
        if c1[k] < min_supp:
            del c1[k]
    return c1

def SortDictionary(dic):
    ndic = {}
    l = sorted(dic.items(), key=lambda kv: kv[1])
    for e in l:
        ndic[e[0]] = e[1]
    return ndic

def calc_hconf(item_set, items_support, support):
    Arr = []
    for item in item_set:
        Arr.append(items_support[item])
    return support/max(Arr)

def HyperCliqueAlgo():
    Result = []
    tManager = TransactionManager.create(Data)
    Candidates = tManager.initial_candidates()
    ItemSupport = {}
    for ItemSet in Candidates:
        ll = list(ItemSet)[0]
        ItemSupport[ll] = tManager.calc_support(ItemSet)
    
    PrevCandidates = Candidates
    size = 1
    
    # Step-1
    relations = set()
    for ItemSet in Candidates:
        support = tManager.calc_support(ItemSet)
        if support < min_supp:
            continue
        candidate_set = frozenset(ItemSet)
        relations.add(candidate_set)
    size += 1
    Candidates = create_next_candidates(relations, size)
    
    # Below code contains step-2 to 4
    while Candidates:
        relations = set()
        for ItemSet in Candidates:
            support = tManager.calc_support(ItemSet)
            hconf = calc_hconf(ItemSet, ItemSupport, support)

            # Prune A
            Subsets = list(combinations(ItemSet, size-1)) 
            Flag = False
            for sub in Subsets:
                if frozenset(sub) not in PrevCandidates:
                    Flag = True
                    break
            if Flag:
                continue
                    
            # Prune B
            # Need to implement
            
            # Prune C
            if support < min_supp or hconf < min_hc:
                continue
            
            candidate_set = frozenset(ItemSet)
            relations.add(candidate_set)
#             print(candidate_set)
            Result.append(candidate_set)
        size += 1
        if size > (MaxEl-MinEl):
            break
        PrevCandidates = Candidates
        Candidates = create_next_candidates(relations, size)
    return Result

In [4]:
Data = LoadData("retail.dat")
N = len(Data)
MaxEl, MinEl = GetMax(Data)
print(MaxEl, MinEl)

16469 0


In [ ]:
MinSupList = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
MinHcList = [0.1, 0.2, 0.5, 0.7, 0.9]
Results = []
Times = []
for min_hc in tqdm(MinHcList):
    temp = []
    temp_time = []
    for min_supp in MinSupList:
        start_time = time.time()
        Result = HyperCliqueAlgo()
        temp_time.append(time.time()-start_time)
        temp.append(len(Result))
    Results.append(temp)
    Times.append(temp_time)
print(Results)
print(Times)

In [ ]:
def HyperCliqueCountGraphs():
    plt.figure()
    for i in range(len(Results)):
        plt.plot(MinSupList, Results[i], label="hconf {}".format(str(MinHcList[i])))
    plt.title('Number of patterns generated by HyperCliqueMiner')
    plt.ylabel('HyperClique Count')
    plt.xlabel('Support Threshold')
    plt.legend()
    plt.show()
    
def HyperCliqueTimeGraph():
    plt.figure()
    for i in range(len(Results)):
        plt.plot(MinSupList, Times[i], label="hconf {}".format(str(MinHcList[i])))
    plt.title('The execution time of hyperclique miner')
    plt.ylabel('Execution Time in Seconds')
    plt.xlabel('Support Threshold')
    plt.legend()
    plt.show()
    
HyperCliqueCountGraphs()
HyperCliqueTimeGraph()